In [ ]:
# import tempfile
# import os

# # Crear un directorio temporal
# temp_dir = tempfile.mkdtemp()

# # Realizar operaciones en el directorio temporal

# # Eliminar el directorio temporal y su contenido
# os.rmdir(temp_dir)


In [1]:
import os

# Cambiar el directorio de trabajo actual
os.chdir("G:\\Mi unidad\\Python\\Proyecto-deteccion-bayas")

# Ahora, las rutas relativas se basarán en el nuevo directorio actual

In [2]:
# Obtener la ruta actual del directorio
current_directory = os.getcwd()
current_directory

'g:\\Mi unidad\\Python\\Proyecto-deteccion-bayas'

In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
!pip install tqdm

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install pyyaml

In [8]:
#!pip install opencv-python==4.7.0.72
!pip install opencv-python
#!conda remove opencv

In [1]:
import cv2
print(cv2.__file__)

c:\Users\cvasquezv\Anaconda3\envs\myenv\lib\site-packages\cv2\__init__.py


In [ ]:
import numpy as np 
import pandas as pd
import os
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from glob import glob
from tqdm.notebook import tqdm
tqdm.pandas()
import random
import json
from sklearn.model_selection import train_test_split
import shutil
import yaml
plt.style.use("ggplot")


In [ ]:
print(cv.__version__)

In [ ]:
!pip install --user roboflow

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="ysLbfsnaroQianBM7EmF")
# project = rf.workspace("inkastats-sac").project("blueberries-detection")
# dataset = project.version(4).download("coco")

In [ ]:
BASE_DIR = "C:/Users/cvasquezv/Proyecto-vigor-defoliacion/input"
annotation_list = glob("C:/Users/cvasquezv/Proyecto-vigor-defoliacion/input/mortalidad/annotations/*.json")

In [ ]:
data = pd.DataFrame()
data["annotation_dir"] = annotation_list

In [ ]:
data

In [ ]:
ruta_completa = "C:/Users/cvasquezv/Proyecto-vigor-defoliacion/input/4712331.FOTO.121358.jpg"
nombre_archivo = os.path.basename(ruta_completa)

print(nombre_archivo)

In [ ]:
img_dir = "C:/Users/cvasquezv/Proyecto-vigor-defoliacion/input../../../input/4712331.FOTO.121358.jpg"
print("Antes de las sustituciones:", img_dir)

img_dir = img_dir.replace("/input../../../input/", "/").replace("/input../../", "/").replace("/input../", "/input/")
print("Después de las sustituciones:", img_dir)

In [ ]:
import pandas as pd
import json
from glob import glob

def read_json_data(jsonfile):
    rows = {"img_dir":[] ,"img_w":[] , "img_h" : [], "sp_type": [] ,"xc":[] , "yc":[], "bb_height" : [] , "bb_width": [] , "bb_left":[] , "bb_top":[]}
    
    # read file
    with open(jsonfile) as json_f:
        json_data = json.load(json_f)

        img_dir = json_data["imagePath"].replace("\\", "/")
        #img_dir = img_dir.split("./input")[1]  # Corregir la construcción de la ruta

        img_dir = img_dir.replace("../../../input/", "/").replace("../../", "/").replace("..", "")
        #img_dir = img_dir.replace("../..", "/")
        
        sp_type = json_data["shapes"][0]["label"]

        img_w = json_data["imageWidth"]
        img_h = json_data["imageHeight"]

        # Normalize Bounding Box
        bb_points = json_data["shapes"][0]["points"]
        x1, y1 = bb_points[0]
        x2, y2 = bb_points[1]

        # Obtén las coordenadas del centro de la bounding box
        xc = (x1 + x2) / (2 * img_w)
        yc = (y1 + y2) / (2 * img_h)

        # Obtén el ancho y alto de la bounding box
        bb_width = abs(x2 - x1) / img_w
        bb_height = abs(y2 - y1) / img_h

        # Normalizar las coordenadas izquierda y superior de la bounding box
        bb_left = min(x1, x2) / img_w
        bb_top = min(y1, y2) / img_h

        rows["img_dir"].append(BASE_DIR + img_dir)
        rows["sp_type"].append(sp_type)
        rows["img_w"].append(img_w)
        rows["img_h"].append(img_h)
        rows["bb_height"].append(bb_height)
        rows["bb_width"].append(bb_width)
        rows["bb_left"].append(bb_left)
        rows["bb_top"].append(bb_top)
        rows["xc"].append(xc)
        rows["yc"].append(yc)

    return rows

data_list = []

for json_file in annotation_list:
    data_list.append(pd.DataFrame(read_json_data(json_file)))

df = pd.concat(data_list, ignore_index=True)
df.head()

In [ ]:
#df = pd.concat(data_list, ignore_index=True)
#print(df)

In [ ]:
#for json_file in tqdm(data["annotation_dir"]):
#    rows = read_json_data(json_file)
    
    # Crea un nuevo DataFrame a partir del diccionario y luego concaténalo con el DataFrame principal
#    df = pd.concat([df, pd.DataFrame.from_dict(rows)], ignore_index=True)

# Ahora df contiene todos los datos combinados


In [ ]:
df = df.reset_index(drop=True)

df['sp_type'] = df['sp_type'].str.strip().replace({"0": "Dead", "1": "Alive"})

In [ ]:
df.head()

In [ ]:
df['img_dir'].head()

In [ ]:
#!pip install openpyxl

In [ ]:
import openpyxl

# Guardar el DataFrame en un archivo Excel
df.to_excel('data info.xlsx', index=False)  # Cambia 'nombre_del_archivo.xlsx' por el nombre que desees

In [ ]:
df.sp_type.value_counts().plot(kind='bar')
plt.show()

## Display Some examples

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def display_random_imgs(df, rows, cols):
    idxs = random.sample(df.index.tolist(), rows * cols)
    fig, ax = plt.subplots(rows, cols, figsize=(10, 10))

    if rows * cols != 1:
        for count, axs in enumerate(ax.flatten()):
            path = df.img_dir.iloc[idxs[count]]
            img = plt.imread(path)

            # Obtén las coordenadas normalizadas de la bounding box
            x = df.bb_left.iloc[idxs[count]]
            y = df.bb_top.iloc[idxs[count]]
            w = df.bb_width.iloc[idxs[count]]
            h = df.bb_height.iloc[idxs[count]]

            # Convierte las coordenadas normalizadas a píxeles
            x_pixel = x * df.img_w.iloc[idxs[count]]
            y_pixel = y * df.img_h.iloc[idxs[count]]
            w_pixel = w * df.img_w.iloc[idxs[count]]
            h_pixel = h * df.img_h.iloc[idxs[count]]

            # Crea el rectángulo
            patch = patches.Rectangle((x_pixel, y_pixel), w_pixel, h_pixel,
                                      linewidth=1, edgecolor='r', facecolor='none')

            axs.imshow(img)
            axs.add_patch(patch)
            axs.axis('off')
            axs.set_title(df.sp_type.iloc[idxs[count]])

    plt.show()

# Llamada a la función
# display_random_imgs(df, 2, 2)


In [ ]:
display_random_imgs(df ,5, 3)

In [ ]:
classes_name = ["Dead","Alive"]
classes_num = [0,1]

df.sp_type = df.sp_type.replace(classes_name , classes_num)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Suponiendo que df es tu DataFrame original

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=42)  # El 25% de train para validación

# Ahora, 'train', 'val' y 'test' contienen los conjuntos de entrenamiento, validación y prueba respectivamente


In [ ]:
import os

# Obtener el directorio actual del script
current_directory = os.getcwd()

# Crear directorios relativos al directorio actual
os.makedirs(os.path.join(current_directory, "working/data/train"), exist_ok=True)
os.makedirs(os.path.join(current_directory, "working/data/valid"), exist_ok=True)
os.makedirs(os.path.join(current_directory, "working/data/test"), exist_ok=True)


In [ ]:
import os
import shutil
from tqdm import tqdm

# Asignar datos a los directorios correspondientes
def add_data_to_folder(file_type, data):
    for index, row in tqdm(data.iterrows(), total=len(data)):
        shutil.copy(row["img_dir"], f"working/data/{file_type}")
        with open(f'working/data/{file_type}/{row.img_dir.split("/")[-1][:-4]}.txt', "w") as f:
            f.write(str(row["sp_type"]))
            f.write(" ")
            f.write(str(row["xc"]))
            f.write(" ")
            f.write(str(row["yc"]))
            f.write(" ")
            f.write(str(row["bb_width"]))
            f.write(" ")
            f.write(str(row["bb_height"]))
            f.write("\n")

In [ ]:
#def add_data_to_folder(file_type, data):
#    for index, row in tqdm(data.iterrows(), total=len(data)):
#        shutil.copy(row["img_dir"], f"working/data/{file_type}")
#        with open(f'working/data/{file_type}/{row.img_dir.split("/")[-1][:-4]}.txt', "w") as f:
#            f.write(str(row["sp_type"]))  # escribir la clase
#            f.write(" ")
#            f.write(str(row["bb_left"]))
#            f.write(" ")
#            f.write(str(row["bb_top"]))
#            f.write(" ")
#            f.write(str(row["bb_width"]))
#            f.write(" ")
#            f.write(str(row["bb_height"]))
#            f.write("\n")

In [ ]:
import random

random.seed(123)  # Establece la semilla de aleatorización

# A continuación, puedes usar funciones de generación de números aleatorios, como random.randint(), etc.


In [ ]:
# Luego puedes llamar a la función para cada conjunto de datos
# Asignar a train
add_data_to_folder("train", train_data)

# Asignar a valid
add_data_to_folder("valid", valid_data)

# Asignar a test
add_data_to_folder("test", test_data)


In [ ]:
# create yml file
#yaml_dict = dict(
#    train = 'C:/Users/cvasquezv/kaggle/working/data/train',
#    val = 'C:/Users/cvasquezv/kaggle/working/data/test',
#    
#    nc    = len(classes_num), # number of classes
#    names = classes_name # classes
#    )

#with open('C:/Users/cvasquezv/kaggle/working/data.yaml', 'w') as outfile:
#    yaml.dump(yaml_dict, outfile, default_flow_style=False)

#%cat C:/Users/cvasquezv/kaggle/working/data.yaml

In [ ]:
# !pip install pyyaml


In [ ]:
import yaml

# Definir la información del YAML
yaml_data = {
    'train': 'C:\\Users\\cvasquezv\\Proyecto-vigor-defoliacion\\working\\data\\train\\',
    'val': 'C:\\Users\\cvasquezv\\Proyecto-vigor-defoliacion\\working\\data\\valid\\',
    #'test': 'C:\\Users\\cvasquezv\\Proyecto-vigor-defoliacion\\working\\data\\test\\',
    'nc': 2,
    'names': ["Dead","Alive"]#,
    # 'num': ["0","1"]
    
}

# Guardar el YAML en un archivo en el directorio working
yaml_file_path = 'C:\\Users\\cvasquezv\\Proyecto-vigor-defoliacion\\working\\data.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False)

print(f"Archivo YAML generado y guardado en: {yaml_file_path}")


In [ ]:
with open('C:/Users/cvasquezv/Proyecto-vigor-defoliacion/working/data.yaml', 'r') as infile:
    yaml_content = infile.read()

print(yaml_content)


In [ ]:
data_path = r'C:\Users\cvasquezv\Proyecto-vigor-defoliacion\working\data\train'

# Obtener la lista de archivos de anotación .txt en el directorio
annotation_files = [f for f in os.listdir(data_path) if f.endswith('.txt')]

# Crear un DataFrame para almacenar la información
df_train = pd.DataFrame(columns=['class'])

# Analizar cada archivo de anotación
for annotation_file in annotation_files:
    annotation_file_path = os.path.join(data_path, annotation_file)

    with open(annotation_file_path, 'r') as f:
        # Cada línea del archivo .txt representa una bounding box
        lines = f.readlines()

        for line in lines:
            # La primera columna en cada línea es el número de clase
            # Puedes adaptar esto según el formato específico de tus archivos de anotación
            class_number = line.split()[0]

            # Agregar la clase al DataFrame
            df_train = pd.concat([df_train, pd.DataFrame({'class': [class_number]})], ignore_index=True)

# Contar la frecuencia de cada clase
class_counts = df_train['class'].value_counts()

# Crear un gráfico de barras para visualizar la distribución de clases
plt.figure(figsize=(10, 6))
class_counts.plot(kind='bar', color='skyblue')
plt.title('Distribución de Clases en Conjunto de Datos YOLO')
plt.xlabel('Número de Clase')
plt.ylabel('Número de Bounding Boxes')
plt.show()

# Imprimir la tabla de frecuencia
print(class_counts)

In [ ]:
df_train.head()

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
def display_images_with_rectangles(data_dir, rows, cols):
    # Obtén la lista de archivos de imágenes y anotaciones
    image_files = [f for f in os.listdir(data_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
    annotation_files = [f for f in os.listdir(data_dir) if f.lower().endswith('.txt')]

    # Filtra las imágenes que tienen anotaciones asociadas
    valid_image_files = [img for img in image_files if f"{os.path.splitext(img)[0]}.txt" in annotation_files]

    # Selecciona aleatoriamente las imágenes a mostrar
    selected_images = random.sample(valid_image_files, rows * cols)

    # Crea el subgráfico
    fig, ax = plt.subplots(rows, cols, figsize=(10, 10))

    if rows * cols != 1:
        for count, axs in enumerate(ax.flatten()):
            # Ruta de la imagen
            img_path = os.path.join(data_dir, selected_images[count])
            img = plt.imread(img_path)

            # Ruta del archivo de anotación
            annotation_path = os.path.join(data_dir, f"{os.path.splitext(selected_images[count])[0]}.txt")

            # Lee las coordenadas del archivo de anotación
            with open(annotation_path, 'r') as annotation_file:
                sp_type, xc, yc, bb_width, bb_height = annotation_file.readline().split()
                xc, yc, bb_width, bb_height = map(float, [xc, yc, bb_width, bb_height])

            # Convierte las coordenadas normalizadas a píxeles
            img_w, img_h = img.shape[1], img.shape[0]
            x_pixel = int(xc * img_w - (bb_width * img_w) / 2)
            y_pixel = int(yc * img_h - (bb_height * img_h) / 2)
            w_pixel = int(bb_width * img_w)
            h_pixel = int(bb_height * img_h)

            # Crea el rectángulo
            patch = patches.Rectangle((x_pixel, y_pixel), w_pixel, h_pixel,
                                      linewidth=1, edgecolor='r', facecolor='none')

            axs.imshow(img)
            axs.add_patch(patch)
            axs.axis('off')
            axs.set_title(f'{sp_type}')

    plt.show()


In [ ]:
# Directorio que contiene los archivos de texto y las imágenes
data_directory = "C:\\Users\\cvasquezv\\Proyecto-vigor-defoliacion\\working\\data\\train"

# Visualizar imágenes aleatorias con bounding boxes
display_images_with_rectangles(data_directory, rows=5, cols=3)

In [ ]:
del df , data

## Yolo v8

In [ ]:
#!pip install -U opencv-python
#!conda install -c conda-forge opencv=4.9


In [4]:
!pip install -U ultralytics

     ---------------------------------------- 0.0/40.2 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.2 kB ? eta -:--:--
     ----------------------------- -------- 30.7/40.2 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 40.2/40.2 kB 318.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/709.5 kB ? eta -:--:--
   -- ------------------------------------ 41.0/709.5 kB 991.0 kB/s eta 0:00:01
   ---- ---------------------------------- 81.9/709.5 kB 919.0 kB/s eta 0:00:01
   ---- ---------------------------------- 81.9/709.5 kB 919.0 kB/s eta 0:00:01
   ----- --------------------------------- 92.2/709.5 kB 655.4 kB/s eta 0:00:01
   ------- ------------------------------ 143.4/709.5 kB 655.8 kB/s eta 0:00:01
   ---------- --------------------------- 204.8/709.5 kB 778.2 kB/s eta 0:00:01
   ------------- ------------------------ 245.8/709.5 kB 885.4 kB/s eta 0:00:01
   ------------- ------------------------ 245.8/709.5 kB 885.4 k

In [ ]:
#!pip install --upgrade pyyaml

In [5]:
pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cuXXX/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cuXXX/torch_stable.html
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
    --------------------------------------- 0.0/2.3 MB 653.6 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/2.3 MB 1.4 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.3 MB 1.9 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/2.3 MB 1.5 MB/s eta 0:00:02
   -------- ------------------------------- 0.5/2.3 MB 2.0 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.3 MB 2.2 MB/s eta 0:00:01
   -------------- ------------------------- 0.9/2.3 MB 2.6 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.3 MB 2.9 MB/s eta 0:00:01
   ------------------ --------

In [8]:
import ultralytics
from IPython import display
display.clear_output()
!yolo checks



Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.1.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Setup complete ✅ (8 CPUs, 39.7 GB RAM, 398.2/476.3 GB disk)

OS                  Windows-10-10.0.19045-SP0
Environment         Windows
Python              3.10.13
Install             pip
RAM                 39.73 GB
CPU                 11th Gen Intel Core(TM) i7-1165G7 2.80GHz
CUDA                None

matplotlib          ✅ 3.8.2>=3.3.0
numpy               ✅ 1.26.2>=1.22.2
opencv-python       ✅ 4.8.1.78>=4.6.0
pillow              ✅ 10.1.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.1.1>=1.8.0
torchvision         ✅ 0.16.1>=0.9.0
tqdm                ✅ 4.66.1>=4.64.0
psutil              ✅ 5.9.0
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 2.1.3>=1.1.4
seaborn             ✅ 0.13.0>=0.11.0


In [9]:
from ultralytics import YOLO

In [ ]:
# Load a model
model = YOLO("yolov8x.pt")  # load a pretrained model (recommended for training)

In [ ]:
batch_size = 20
imgsz = 640

In [ ]:
# Use the model
results = model.train(data="C:/Users/cvasquezv/Proyecto-vigor-defoliacion/working/data.yaml",
                      epochs=250,
                      batch = batch_size,
                      imgsz=imgsz)  # train the model

In [ ]:
# Ruta al directorio que contiene las imágenes
ruta_imagenes = "working\\data\\test\\"

In [ ]:
# Obtener la lista de archivos de imagen en el directorio
image_files = [f for f in os.listdir(ruta_imagenes) if f.endswith((".jpg", ".jpeg", ".png"))]

# Construir rutas completas de las imágenes
image_paths = [os.path.join(ruta_imagenes, file) for file in image_files]

# Ahora, image_paths contiene las rutas completas de las imágenes
print(image_paths)

In [ ]:
from PIL import Image

# Lista de rutas a las nuevas imágenes
#image_paths = ['working\\data\\test\\002b37ac08e1.jpg',
#               'working\\data\\test\\002da3868a1a.jpg',
#               'working\\data\\test\\0036742ac4ec.jpg']
image_paths

In [ ]:
# Cargar las imágenes
images = [Image.open(path) for path in image_paths]

In [ ]:
# Realizar inferencias en las imágenes
inference = model.predict(images)


In [ ]:
inference

In [ ]:
print(dir(inference))

In [ ]:
from PIL import Image

# Itera a través de las rutas de las imágenes
for image_path in image_paths:
    # Carga la imagen
    image = Image.open(image_path)
    
    # Preprocesa la imagen según sea necesario
    
    # Realiza la inferencia
    results = model.predict(image,
                            save=True,
                            save_crop=True,
                            save_txt=True,
                            imgsz=640,
                            conf=0.1,
                            max_det=300)
    
    # Postprocesa los resultados y visualiza o almacena la salida según tus necesidades

In [ ]:
from PIL import Image

# Itera a través de las rutas de las imágenes
for image_path in image_paths:
    # Carga la imagen
    image = Image.open(image_path)
    
    # Realiza la inferencia
    results = model.predict(image)
    
    # Show the results for this image
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        im.show()  # show image


In [ ]:
#!pip install ultralytics==8.0.175
#!pip install ultralytics
#!pip install ultralytics==8.0.213

In [ ]:
#!pip install ruamel.yaml

In [ ]:
#!conda update -n myenv opencv-python

In [6]:
#import sys
#sys.path.append(r'C:\Users\cvasquezv\Anaconda3\envs\myenv\Lib\site-packages')

import cv2
print(cv2.__version__)

from ultralytics import YOLO

4.8.0


In [ ]:
# Load the YOLOv8 model
model = YOLO("G:/Mi unidad/Python/Proyecto-vigor-defoliacion/runs/detect/train8/weights/best.pt/")

## Ver inferencias del modelo

In [ ]:
# Open the video file
video_path = "8.mp4"
cap = cv2.VideoCapture(video_path)

screen_width = 800  # Reemplaza con la resolución de tu pantalla
screen_height = 600  # Reemplaza con la resolución de tu pantalla


In [ ]:
# Obtener las propiedades del video (ancho, alto, fps)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Porcentaje de reducción de tamaño de la ventana
reduction_percentage = 40

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the original frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Reduce the size of the frame for display
        small_frame = cv2.resize(annotated_frame, (int(width * reduction_percentage / 100), int(height * reduction_percentage / 100)))

        # Display the annotated frame with reduced window size
        cv2.imshow("YOLOv8 Inference", small_frame)

        # Reduce the delay for higher FPS
        cv2.waitKey(int(1000 / fps))  # Set a delay based on the original video's FPS

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break


In [ ]:
cap.release()
cv2.destroyAllWindows()

## Guardar video con conteo y dimensiones reales

In [5]:
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2
import numpy as np

# Open the video file
video_path = "video1.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Percentage reduction of window size
reduction_percentage = 40

# Define region points in a vertical orientation
region_points = [(w*0.40, 0), (w*0.40, h), 
                 (w*0.50, h), (w*0.50, 0)]  # Ajusta las coordenadas según tus preferencias

classes_to_count = [0, 1]

# Video writer
video_writer = cv2.VideoWriter("object_counting_output.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))


# Init Object Counter
counter = object_counter.ObjectCounter()
# Ejemplo de reinicio manual del contador
counter.counter_variable = 0

counter.set_args(view_img=False,
                 reg_pts=region_points,
                 #view_in_counts = True,
                 #view_out_counts = True,
                 classes_names=model.model.names,
                 draw_tracks=True
                 )

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("El frame del video está vacío o el procesamiento del video se ha completado correctamente.")
        break
    tracks = model.track(im0, persist=True, show=False, classes=classes_to_count)

    # Inicia el conteo y obtén el número de objetos contados por clase
    counts_by_class = counter.start_counting(im0, tracks)

    # Reduce el tamaño de la imagen para su visualización
    small_frame = cv2.resize(im0, (int(w * reduction_percentage / 100), int(h * reduction_percentage / 100)))

    # Escribe el frame en el archivo de video
    video_writer.write(im0)

    # Muestra la imagen reducida
    cv2.imshow("Object Counting", small_frame)

    # Rompe el bucle si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

AssertionError: Error reading video file

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

## Conteo y tracker multiclase con Yolov8

In [ ]:
import cv2
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import csv


model = YOLO("G:/Mi unidad/Python/Proyecto-vigor-defoliacion/runs/detect/train8/weights/best.pt/")

# Open the video file
video_path = "8.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Percentage reduction of window size
reduction_percentage = 40

# Define region points in a vertical orientation
region_points = [(w*0.40, 0), (w*0.40, h), 
                 (w*0.50, h), (w*0.50, 0)]  # Ajusta las coordenadas según tus preferencias


classes_to_count = [0, 1]

# Clases a contar (Dead y Alive)
classes_to_count_dead = [0]  # Reemplaza con el índice correcto de la clase "Dead"
classes_to_count_alive = [1]  # Reemplaza con el índice correcto de la clase "Alive"

# Video writer
video_writer = cv2.VideoWriter("object_counting_multiclass.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

# Init Object Counters for each class
counter_dead = object_counter.ObjectCounter()
counter_dead.counter_variable = 0
counter_dead.set_args(view_img=False,
                      view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                      view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                      reg_pts=region_points,
                      classes_names=model.model.names,
                      draw_tracks=True)

counter_alive = object_counter.ObjectCounter()
counter_alive.counter_variable = 0
counter_alive.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.model.names,
                       draw_tracks=True)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("El frame del video está vacío o el procesamiento del video se ha completado correctamente.")
        break

    # Inicia el conteo y obtén el número de objetos contados por clase
    #tracks_dead = model.track(im0, persist=True, show=False, classes=classes_to_count_dead)
    #counts_dead = counter_dead.start_counting(im0, tracks_dead)

    #tracks_alive = model.track(im0, persist=True, show=False, classes=classes_to_count_alive)
    #counts_alive = counter_alive.start_counting(im0, tracks_alive)

    # Create a dictionary to map class labels to counters
    counters_by_class = {"Dead": counter_dead, "Alive": counter_alive}

    # Inicia el conteo y obtén el número de objetos contados por clase
    tracks_dead = model.track(im0, persist=True, show=False, classes=classes_to_count_dead)
    counts_dead = counters_by_class["Dead"].start_counting(im0, tracks_dead)

    tracks_alive = model.track(im0, persist=True, show=False, classes=classes_to_count_alive)
    counts_alive = counters_by_class["Alive"].start_counting(im0, tracks_alive)

    # Dead Count
    text_dead = f'Dead: {counter_dead.out_counts}'
    font_dead = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_dead = 2
    font_thickness_dead = 4
    font_color_dead = (0, 0, 255)
    background_color_dead = (0, 0, 0)

    # Get the size of the text
    (text_width_dead, text_height_dead), _ = cv2.getTextSize(text_dead, font_dead, font_scale_dead, font_thickness_dead)

    # Set the position for the text
    text_position_dead = (10, 60)

    # Create a black background for the text
    background_rect_dead = (text_position_dead[0], text_position_dead[1] - text_height_dead, text_width_dead, text_height_dead)
    cv2.rectangle(im0, background_rect_dead, background_color_dead, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_dead, text_position_dead, font_dead, font_scale_dead, font_color_dead, font_thickness_dead, cv2.LINE_AA)

    # Alive Count
    text_alive = f'Alive: {counter_alive.out_counts}'
    font_alive = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_alive = 2
    font_thickness_alive = 4
    font_color_alive = (0, 255, 0)
    background_color_alive = (0, 0, 0)

    # Get the size of the text
    (text_width_alive, text_height_alive), _ = cv2.getTextSize(text_alive, font_alive, font_scale_alive, font_thickness_alive)

    # Set the position for the text
    text_position_alive = (10, 110)

    # Create a black background for the text
    background_rect_alive = (text_position_alive[0], text_position_alive[1] - text_height_alive, text_width_alive, text_height_alive)
    cv2.rectangle(im0, background_rect_alive, background_color_alive, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_alive, text_position_alive, font_alive, font_scale_alive, font_color_alive, font_thickness_alive, cv2.LINE_AA)

    # Añade el texto con los conteos a la imagen
    #cv2.putText(im0, f'Dead: {counter_dead.out_counts}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    #cv2.putText(im0, f'Alive: {counter_alive.out_counts}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Reduce el tamaño de la imagen para su visualización
    small_frame = cv2.resize(im0, (int(w * reduction_percentage / 100), int(h * reduction_percentage / 100)))
    
    # Escribe el frame en el archivo de video
    video_writer.write(im0)

    # Muestra la imagen reducida
    cv2.imshow("Object Counting", small_frame)

    # Rompe el bucle si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

## Conteo, Tracker y Registro de múltiples clases

In [4]:
from IPython import display
display.clear_output()
 
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.1.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Setup complete ✅ (8 CPUs, 39.7 GB RAM, 399.0/476.3 GB disk)


In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
import cv2
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import csv
import pandas as pd
import openpyxl

model = YOLO("G:/Mi unidad/Python/Proyecto-deteccion-bayas/runs/detect/train4/weights/best.pt/")
# model.to('cpu')

# Export the model
# model.export(format='openvino')  # creates 'yolov8n_openvino_model/'

# Load the exported OpenVINO model
# ov_model = YOLO('G:/Mi unidad/Python/Proyecto-deteccion-bayas/runs/detect/train4/weights/best.onnx/')

In [7]:
# Open the video file
video_path = "video1.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Percentage reduction of window size
reduction_percentage = 60

# Define region points in a vertical orientation
region_points = [(w*0.40, 0), (w*0.40, h), 
                 (w*0.50, h), (w*0.50, 0)]  # Ajusta las coordenadas según tus preferencias

classes_to_count = [0, 1, 2, 3, 4, 5, 6, 7]

# Clases a contar 
classes_to_count_0 = [0]  # Reemplaza con el índice correcto de la clase "Closed flower"
classes_to_count_1 = [1]  # Reemplaza con el índice correcto de la clase "Open flower"
classes_to_count_2 = [2]  # Reemplaza con el índice correcto de la clase "Set fruit"
classes_to_count_3 = [3]  # Reemplaza con el índice correcto de la clase "Green fruit"
classes_to_count_4 = [4]  # Reemplaza con el índice correcto de la clase "Partially green fruit"
classes_to_count_5 = [5]  # Reemplaza con el índice correcto de la clase "Blushing fruit"
classes_to_count_6 = [6]  # Reemplaza con el índice correcto de la clase "Pink fruit"
classes_to_count_7 = [7]  # Reemplaza con el índice correcto de la clase "Blue fruit"

# Video writer
video_writer = cv2.VideoWriter("object_counting_multiclass.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

# Init Object Counters for each class
counter_0 = object_counter.ObjectCounter()
counter_0.counter_variable = 0
counter_0.set_args(view_img=False,
                      view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                      view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                      reg_pts=region_points,
                      classes_names=model.names,
                      draw_tracks=True)

counter_1 = object_counter.ObjectCounter()
counter_1.counter_variable = 0
counter_1.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.names,
                       draw_tracks=True)

counter_2 = object_counter.ObjectCounter()
counter_2.counter_variable = 0
counter_2.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.names,
                       draw_tracks=True)

counter_3 = object_counter.ObjectCounter()
counter_3.counter_variable = 0
counter_3.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.names,
                       draw_tracks=True)

counter_4 = object_counter.ObjectCounter()
counter_4.counter_variable = 0
counter_4.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.names,
                       draw_tracks=True)

counter_5 = object_counter.ObjectCounter()
counter_5.counter_variable = 0
counter_5.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.names,
                       draw_tracks=True)

counter_6 = object_counter.ObjectCounter()
counter_6.counter_variable = 0
counter_6.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.names,
                       draw_tracks=True)

counter_7 = object_counter.ObjectCounter()
counter_7.counter_variable = 0
counter_7.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.names,
                       draw_tracks=True)

counter_all = object_counter.ObjectCounter()
counter_all.counter_variable = 0
counter_all.set_args(view_img=False,
                     view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                     view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                     reg_pts=region_points,
                     classes_names=model.names,
                     draw_tracks=False)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("El frame del video está vacío o el procesamiento del video se ha completado correctamente.")
        break

    # Create a dictionary to map class labels to counters
    counters_by_class = {
        "Closed flower": counter_0,
        "Open flower": counter_1,
        "Set fruit": counter_2,
        "Green fruit": counter_3,
        "Partially green fruit": counter_4,
        "Blushing fruit": counter_5,
        "Pink fruit": counter_6,
        "Blue fruit": counter_7,
        "All": counter_all
    }
    
    # Inicia el conteo y obtén el número de objetos contados por clase
    tracks_0 = model.track(im0, persist=True, show=False, classes=classes_to_count_0,
                               conf = 0.3, iou = 0.9, verbose = True)
    counts_0 = counters_by_class["Closed flower"].start_counting(im0, tracks_0)

    tracks_1 = model.track(im0, persist=True, show=False, classes=classes_to_count_1,
                                conf = 0.3, iou = 0.9, verbose = True)
    counts_1 = counters_by_class["Open flower"].start_counting(im0, tracks_1)

    tracks_2 = model.track(im0, persist=True, show=False, classes=classes_to_count_2,
                                conf = 0.3, iou = 0.9, verbose = True)
    counts_2 = counters_by_class["Set fruit"].start_counting(im0, tracks_2)
    
    tracks_3 = model.track(im0, persist=True, show=False, classes=classes_to_count_3,
                                conf = 0.3, iou = 0.9, verbose = True)
    counts_3 = counters_by_class["Green fruit"].start_counting(im0, tracks_3)

    tracks_4 = model.track(im0, persist=True, show=False, classes=classes_to_count_4,
                                conf = 0.3, iou = 0.9, verbose = True)
    counts_4 = counters_by_class["Partially green fruit"].start_counting(im0, tracks_4)

    tracks_5 = model.track(im0, persist=True, show=False, classes=classes_to_count_5,
                                conf = 0.3, iou = 0.9, verbose = True)
    counts_5 = counters_by_class["Blushing fruit"].start_counting(im0, tracks_5)

    tracks_6 = model.track(im0, persist=True, show=False, classes=classes_to_count_6,
                                conf = 0.3, iou = 0.9, verbose = True)
    counts_6 = counters_by_class["Pink fruit"].start_counting(im0, tracks_6)

    tracks_7 = model.track(im0, persist=True, show=False, classes=classes_to_count_7,
                                conf = 0.3, iou = 0.9, verbose = True)
    counts_7 = counters_by_class["Blue fruit"].start_counting(im0, tracks_7)

    tracks_all = model.track(im0, persist=True, show=False,
                                conf = 0.3, iou = 0.9, verbose = True)
    counts_all = counters_by_class["All"].start_counting(im0, tracks_all)

    # Iniciar el conteo y obtener el número de objetos contados por clase

    # Closed flower Count
    text_0 = f'Closed flower: {counter_0.out_counts}'
    font_0 = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_0 = 2
    font_thickness_0 = 4
    font_color_0 = (173, 216, 230)
    background_color_0 = (0, 0, 0)

    # Get the size of the text
    (text_width_0, text_height_0), _ = cv2.getTextSize(text_0, font_0, font_scale_0, font_thickness_0)

    # Set the position for the text
    text_position_0 = (10, 60)

    # Create a black background for the text
    background_rect_0 = (text_position_0[0], text_position_0[1] - text_height_0, text_width_0, text_height_0)
    cv2.rectangle(im0, background_rect_0, background_color_0, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_0, text_position_0, font_0, font_scale_0, font_color_0, font_thickness_0, cv2.LINE_AA)

    # Open flower Count
    text_1 = f'Open flower: {counter_1.out_counts}'
    font_1 = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_1 = 2
    font_thickness_1 = 4
    font_color_1 = (0, 100, 0)
    background_color_1 = (0, 0, 0)

    # Get the size of the text
    (text_width_1, text_height_1), _ = cv2.getTextSize(text_1, font_1, font_scale_1, font_thickness_1)

    # Set the position for the text
    text_position_1 = (10, 110)

    # Create a black background for the text
    background_rect_1 = (text_position_1[0], text_position_1[1] - text_height_1, text_width_1, text_height_1)
    cv2.rectangle(im0, background_rect_1, background_color_1, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_1, text_position_1, font_1, font_scale_1, font_color_1, font_thickness_1, cv2.LINE_AA)

    # Set fruit Count
    text_2 = f'Set fruit: {counter_2.out_counts}'
    font_2 = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_2 = 2
    font_thickness_2 = 4
    font_color_2 = (128, 128, 0)
    background_color_2 = (0, 0, 0)

    # Get the size of the text
    (text_width_2, text_height_2), _ = cv2.getTextSize(text_2, font_2, font_scale_2, font_thickness_2)

    # Set the position for the text
    text_position_2 = (10, 160)

    # Create a black background for the text
    background_rect_2 = (text_position_2[0], text_position_2[1] - text_height_2, text_width_2, text_height_2)
    cv2.rectangle(im0, background_rect_2, background_color_2, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_2, text_position_2, font_2, font_scale_2, font_color_2, font_thickness_2, cv2.LINE_AA)    

    # Green Count
    text_3 = f'Green: {counter_3.out_counts}'
    font_3 = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_3 = 2
    font_thickness_3 = 4
    font_color_3 = (0, 128, 0)
    background_color_3 = (0, 0, 0)

    # Get the size of the text
    (text_width_3, text_height_3), _ = cv2.getTextSize(text_3, font_3, font_scale_3, font_thickness_3)

    # Set the position for the text
    text_position_3 = (10, 210)

    # Create a black background for the text
    background_rect_3 = (text_position_3[0], text_position_3[1] - text_height_3, text_width_3, text_height_3)
    cv2.rectangle(im0, background_rect_3, background_color_3, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_3, text_position_3, font_3, font_scale_3, font_color_3, font_thickness_3, cv2.LINE_AA)

    # Partially green Count
    text_4 = f'Partially green: {counter_4.out_counts}'
    font_4 = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_4 = 2
    font_thickness_4 = 4
    font_color_4 = (85, 107, 47)
    background_color_4 = (0, 0, 0)

    # Get the size of the text
    (text_width_4, text_height_4), _ = cv2.getTextSize(text_4, font_4, font_scale_4, font_thickness_4)

    # Set the position for the text
    text_position_4 = (10, 260)

    # Create a black background for the text
    background_rect_4 = (text_position_4[0], text_position_4[1] - text_height_4, text_width_4, text_height_4)
    cv2.rectangle(im0, background_rect_4, background_color_4, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_4, text_position_4, font_4, font_scale_4, font_color_4, font_thickness_4, cv2.LINE_AA)

    # Blushing Count
    text_5 = f'Blushing: {counter_5.out_counts}'
    font_5 = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_5 = 2
    font_thickness_5 = 4
    font_color_5 = (255, 182, 193)
    background_color_5 = (0, 0, 0)

    # Get the size of the text
    (text_width_5, text_height_5), _ = cv2.getTextSize(text_5, font_5, font_scale_5, font_thickness_5)

    # Set the position for the text
    text_position_5 = (10, 310)

    # Create a black background for the text
    background_rect_5 = (text_position_5[0], text_position_5[1] - text_height_5, text_width_5, text_height_5)
    cv2.rectangle(im0, background_rect_5, background_color_5, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_5, text_position_5, font_5, font_scale_5, font_color_5, font_thickness_5, cv2.LINE_AA)

    # Pink Count
    text_6 = f'Pink: {counter_6.out_counts}'
    font_6 = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_6 = 2
    font_thickness_6 = 4
    font_color_6 = (255, 105, 180)
    background_color_6 = (0, 0, 0)

    # Get the size of the text
    (text_width_6, text_height_6), _ = cv2.getTextSize(text_6, font_6, font_scale_6, font_thickness_6)

    # Set the position for the text
    text_position_6 = (10, 360)

    # Create a black background for the text
    background_rect_6 = (text_position_6[0], text_position_6[1] - text_height_6, text_width_6, text_height_6)
    cv2.rectangle(im0, background_rect_6, background_color_6, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_6, text_position_6, font_6, font_scale_6, font_color_6, font_thickness_6, cv2.LINE_AA)

    # Blue Count
    text_7 = f'Blue: {counter_7.out_counts}'
    font_7 = cv2.FONT_HERSHEY_SIMPLEX
    font_scale_7 = 2
    font_thickness_7 = 4
    font_color_7 = (139, 0, 0)
    background_color_7 = (0, 0, 0)

    # Get the size of the text
    (text_width_7, text_height_7), _ = cv2.getTextSize(text_7, font_7, font_scale_7, font_thickness_7)

    # Set the position for the text
    text_position_7 = (10, 410)

    # Create a black background for the text
    background_rect_7 = (text_position_7[0], text_position_7[1] - text_height_7, text_width_7, text_height_7)
    cv2.rectangle(im0, background_rect_7, background_color_7, thickness=cv2.FILLED)

    # Put the text on the image
    cv2.putText(im0, text_7, text_position_7, font_7, font_scale_7, font_color_7, font_thickness_7, cv2.LINE_AA)

    # Guardar conteos en stream en un archivo Excel
    data_0 = {'id': counter_0.counting_list, 'class': 'Closed flower'}
    df_0 = pd.DataFrame(data_0)

    data_1 = {'id': counter_1.counting_list, 'class': 'Open flower'}
    df_1 = pd.DataFrame(data_1)

    data_2 = {'id': counter_2.counting_list, 'class': 'Set fruit'}
    df_2 = pd.DataFrame(data_2)

    data_3 = {'id': counter_3.counting_list, 'class': 'Green'}
    df_3 = pd.DataFrame(data_3)

    data_4 = {'id': counter_4.counting_list, 'class': 'Partially green'}
    df_4 = pd.DataFrame(data_4)

    data_5 = {'id': counter_5.counting_list, 'class': 'Blushing'}
    df_5 = pd.DataFrame(data_5)

    data_6 = {'id': counter_6.counting_list, 'class': 'Pink'}
    df_6 = pd.DataFrame(data_6)

    data_7 = {'id': counter_7.counting_list, 'class': 'Blue'}
    df_7 = pd.DataFrame(data_7)

    df_combined = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7], ignore_index=True)
    df_combined = df_combined.sort_values(by='id')

    excel_filename = 'combined_counts.xlsx'
    df_combined.to_excel(excel_filename, index=False)

    # Reduce el tamaño de la imagen para su visualización
    small_frame = cv2.resize(im0, (int(w * reduction_percentage / 100), int(h * reduction_percentage / 100)))
    
    # Escribe el frame en el archivo de video
    video_writer.write(im0)

    # Muestra la imagen reducida
    cv2.imshow("Object Counting", small_frame)

    # Rompe el bucle si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

Region Counter Initiated.
Region Counter Initiated.
Region Counter Initiated.
Region Counter Initiated.
Region Counter Initiated.
Region Counter Initiated.
Region Counter Initiated.
Region Counter Initiated.
Region Counter Initiated.

0: 640x384 (no detections), 976.7ms
Speed: 10.1ms preprocess, 976.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 952.1ms
Speed: 1.4ms preprocess, 952.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Set fruit, 951.2ms
Speed: 0.0ms preprocess, 951.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 949.6ms
Speed: 0.0ms preprocess, 949.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 942.1ms
Speed: 8.0ms preprocess, 942.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 897.7ms
Speed: 0.0ms preprocess, 897.7ms inference, 1.0ms postpro

KeyboardInterrupt: 

: 

In [1]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

In [ ]:
import pandas as pd
import openpyxl

# Datos de counter_dead.counting_list
data_dead = {'id': counter_dead.counting_list, 'class': 'dead'}
df_dead = pd.DataFrame(data_dead)

# Datos de counter_alive.counting_list
data_alive = {'id': counter_alive.counting_list, 'class': 'alive'}
df_alive = pd.DataFrame(data_alive)

# Unir ambos dataframes
df_combined = pd.concat([df_dead, df_alive], ignore_index=True)

# Ordenar por ID
df_combined = df_combined.sort_values(by='id')

# Guardar en un archivo Excel
excel_filename = 'combined_counts.xlsx'
df_combined.to_excel(excel_filename, index=False)


In [ ]:
#!pip install openpyxl

In [ ]:
#!pip install shapely
!conda activate myenv && pip install shapely

In [ ]:
import shapely

print(shapely.__file__)

In [ ]:
!conda activate myenv && pip install lapx

In [148]:
#tracker1
print(dir(counts_all))

['T', '__abs__', '__add__', '__and__', '__array__', '__array_finalize__', '__array_function__', '__array_interface__', '__array_prepare__', '__array_priority__', '__array_struct__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__class_getitem__', '__complex__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__divmod__', '__dlpack__', '__dlpack_device__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__ilshift__', '__imatmul__', '__imod__', '__imul__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__irshift__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lshift__', '__lt__', '__matmul__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__',

In [152]:
# Objeto counts_dead
print("Información de cada elemento de la lista:")
for atributo in dir(counter_all):
    # Utiliza getattr() para obtener el valor del atributo
    valor_atributo = getattr(counter_all, atributo)
    # Imprime el nombre del atributo y su valor
    print(f"{atributo}: {valor_atributo}")

Información de cada elemento de la lista:
__class__: <class 'ultralytics.solutions.object_counter.ObjectCounter'>
__delattr__: <method-wrapper '__delattr__' of ObjectCounter object at 0x0000022284E42B90>
__dict__: {'is_drawing': False, 'selected_point': None, 'reg_pts': [(432.0, 0), (432.0, 1920), (540.0, 1920), (540.0, 0)], 'line_dist_thresh': 15, 'counting_region': <POLYGON ((432 0, 432 1920, 540 1920, 540 0, 432 0))>, 'region_color': (255, 0, 255), 'region_thickness': 5, 'im0': array([[[252, 250, 235],
        [252, 250, 235],
        [253, 251, 236],
        ...,
        [206, 178, 159],
        [206, 178, 159],
        [206, 178, 159]],

       [[252, 250, 235],
        [252, 250, 235],
        [253, 251, 236],
        ...,
        [204, 176, 157],
        [204, 176, 157],
        [204, 176, 157]],

       [[252, 250, 235],
        [252, 250, 235],
        [253, 251, 236],
        ...,
        [205, 177, 158],
        [205, 177, 158],
        [205, 177, 158]],

       ...,

      

In [ ]:
# Acceder a los conteos
in_count = counter_dead.in_counts
in_count


In [ ]:
out_count = counter_dead.out_counts
out_count

In [153]:
clases_detectadas = counter_dead.track_history

# Obtener solo los ID de cada objeto detectado
ids_detectados = list(clases_detectadas.keys())

# Imprimir los ID de los objetos detectados
print(ids_detectados)

[1, 2, 3]


In [154]:
counter_dead.counting_list

[1, 2, 3]

In [155]:
counter_alive.counting_list

[5, 4, 8, 11, 17]

## Guardar video en dimensiones personalizadas

In [ ]:
# Nombre del archivo de video de entrada
video_path = "8.mp4"

# Nombre del archivo de video de salida (formato MP4)
output_path = "output.mp4"

# Inicializar el lector de video
cap = cv2.VideoCapture(video_path)

# Obtener las propiedades del video (ancho, alto, fps)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Inicializar el escritor de video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Resize the frame to a smaller resolution (optional)
        frame = cv2.resize(frame, (600, 800))  # Adjust the new_width and new_height

        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Write the annotated frame to the output video
        out.write(annotated_frame)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Reduce the delay for higher FPS
        # cv2.waitKey(1)  # Set a delay for approximately 60 FPS

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break 

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

## Guardar video en dimensiones reales

In [ ]:
# Inicializar el lector de video
cap = cv2.VideoCapture(video_path)

# Obtener las propiedades del video (ancho, alto, fps)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [ ]:
width

In [ ]:
height

In [ ]:
# Inicializar el lector de video
cap = cv2.VideoCapture(video_path)

# Obtener las propiedades del video (ancho, alto, fps)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Inicializar el escritor de video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Escribir el frame procesado en el video de salida
        out.write(annotated_frame)

    else:
        # Break the loop if the end of the video is reached
        break 

# Liberar recursos
cap.release()
out.release()

print(f"Video de salida guardado en: {output_path}")

## Inferencias en tiempo real

In [ ]:
# Establecer la dirección IP y el puerto proporcionados por la aplicación "IP Webcam"
ip_address = "http://10.4.41.24:8080"  # Ruta base proporcionada por la aplicación

# URL completa para acceder al video en vivo
video_url = f"{ip_address}/video"

# Obtener la resolución de tu pantalla
screen_width = 1920  # Reemplaza con la resolución de tu pantalla
screen_height = 1080  # Reemplaza con la resolución de tu pantalla

# Crear un objeto VideoCapture para acceder al video de la cámara del teléfono
cap = cv2.VideoCapture(video_url)

while True:
    # Capturar un fotograma de la cámara del teléfono
    ret, frame = cap.read()

    if success:
        # Resize the frame to a smaller resolution (optional)
        frame = cv2.resize(frame, (screen_width, screen_height))  # Adjust the new_width and new_height

        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Reduce the delay for higher FPS
        # cv2.waitKey(1)  # Set a delay for approximately 60 FPS

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break




In [ ]:
# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()

## Conteo, Tracking, Registro en tiempo real

In [ ]:
import cv2
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import csv
import pandas as pd
import openpyxl
import threading

model = YOLO("G:/Mi unidad/Python/Proyecto-vigor-defoliacion/runs/detect/train8/weights/best.pt/")

excel_filename = 'combined_counts_test.xlsx'

# Establecer la dirección IP y el puerto proporcionados por la aplicación "IP Webcam"
ip_address = "https://172.20.10.12:8080"  # Ruta base proporcionada por la aplicación

# URL completa para acceder al video en vivo
video_url = f"{ip_address}/video"
#video_url = 0

cap = cv2.VideoCapture(video_url)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Porcentaje de reducción de tamaño de la ventana
reduction_percentage = 0.4

# Define region points in a vertical orientation
region_points = [(w*0.40, 0), (w*0.40, h), 
                 (w*0.50, h), (w*0.50, 0)]  # Ajusta las coordenadas según tus preferencias

classes_to_count = [0, 1]

# Clases a contar (Dead y Alive)
classes_to_count_dead = [0]  # Reemplaza con el índice correcto de la clase "Dead"
classes_to_count_alive = [1]  # Reemplaza con el índice correcto de la clase "Alive"

# Video writer
video_writer = cv2.VideoWriter("object_counting_multiclass_test2.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

# Init Object Counters for each class
counter_dead = object_counter.ObjectCounter()
counter_dead.counter_variable = 0
counter_dead.set_args(view_img=False,
                      view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                      view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                      reg_pts=region_points,
                      classes_names=model.model.names,
                      draw_tracks=True)

counter_alive = object_counter.ObjectCounter()
counter_alive.counter_variable = 0
counter_alive.set_args(view_img=False,
                       view_in_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia adentro
                       view_out_counts=False,  # Configura esto a False para no mostrar etiquetas de conteo hacia afuera
                       reg_pts=region_points,
                       classes_names=model.model.names,
                       draw_tracks=True)

df_combined = pd.DataFrame(columns=['id', 'class'])  # DataFrame compartido entre hilos

def procesar_video():
    global df_combined

    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            print("El frame del video está vacío o el procesamiento del video se ha completado correctamente.")
            break

        counters_by_class = {"Dead": counter_dead, "Alive": counter_alive}
        
        tracks_dead = model.track(im0, persist=True, show=False, classes=classes_to_count_dead)
        counts_dead = counters_by_class["Dead"].start_counting(im0, tracks_dead)

        tracks_alive = model.track(im0, persist=True, show=False, classes=classes_to_count_alive)
        counts_alive = counters_by_class["Alive"].start_counting(im0, tracks_alive)

        # Dead Count
        text_dead = f'Dead: {counter_dead.out_counts}'
        font_dead = cv2.FONT_HERSHEY_SIMPLEX
        font_scale_dead = 2
        font_thickness_dead = 4
        font_color_dead = (0, 0, 255)
        background_color_dead = (0, 0, 0)

        # Get the size of the text
        (text_width_dead, text_height_dead), _ = cv2.getTextSize(text_dead, font_dead, font_scale_dead, font_thickness_dead)

        # Set the position for the text
        text_position_dead = (10, 60)

        # Create a black background for the text
        background_rect_dead = (text_position_dead[0], text_position_dead[1] - text_height_dead, text_width_dead, text_height_dead)
        cv2.rectangle(im0, background_rect_dead, background_color_dead, thickness=cv2.FILLED)

        # Put the text on the image
        cv2.putText(im0, text_dead, text_position_dead, font_dead, font_scale_dead, font_color_dead, font_thickness_dead, cv2.LINE_AA)

        # Alive Count
        text_alive = f'Alive: {counter_alive.out_counts}'
        font_alive = cv2.FONT_HERSHEY_SIMPLEX
        font_scale_alive = 2
        font_thickness_alive = 4
        font_color_alive = (0, 255, 0)
        background_color_alive = (0, 0, 0)

        # Get the size of the text
        (text_width_alive, text_height_alive), _ = cv2.getTextSize(text_alive, font_alive, font_scale_alive, font_thickness_alive)

        # Set the position for the text
        text_position_alive = (10, 110)

        # Create a black background for the text
        background_rect_alive = (text_position_alive[0], text_position_alive[1] - text_height_alive, text_width_alive, text_height_alive)
        cv2.rectangle(im0, background_rect_alive, background_color_alive, thickness=cv2.FILLED)

        # Put the text on the image
        cv2.putText(im0, text_alive, text_position_alive, font_alive, font_scale_alive, font_color_alive, font_thickness_alive, cv2.LINE_AA)

        data_dead = {'id': counter_dead.counting_list, 'class': 'dead'}
        df_dead = pd.DataFrame(data_dead)

        data_alive = {'id': counter_alive.counting_list, 'class': 'alive'}
        df_alive = pd.DataFrame(data_alive)

        df_combined = pd.concat([df_dead, df_alive], ignore_index=True)
        df_combined = df_combined.sort_values(by='id')

        df_combined.to_excel(excel_filename, index=False)

        small_frame = cv2.resize(im0, (int(w * reduction_percentage / 100), int(h * reduction_percentage / 100)))
        video_writer.write(im0)

        cv2.imshow("Object Counting", small_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

# Inicia el hilo para procesar el video
video_thread = threading.Thread(target=procesar_video)
video_thread.start()

# Espera a que el hilo de video termine antes de cerrar la aplicación
video_thread.join()

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
from time import time

In [ ]:
# Establecer la dirección IP y el puerto proporcionados por la aplicación "IP Webcam"
ip_address = "http://10.4.41.24:8080"  # Ruta base proporcionada por la aplicación

# URL completa para acceder al video en vivo
video_url = f"{ip_address}/video"

# Crear un objeto VideoCapture para acceder al video de la cámara del teléfono
cap = cv2.VideoCapture(video_url)

# Obtener la resolución del fotograma de la cámara
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Inicializar el tiempo de referencia para el control de velocidad
start_time = time()

# Velocidad objetivo en FPS
target_fps = 10  # Puedes ajustar esto a la velocidad deseada

In [ ]:
while True:
    # Capturar un fotograma de la cámara del teléfono
    ret, frame = cap.read()

    if not ret:
        break  # Salir del bucle si no se puede capturar un fotograma

    # Run YOLOv8 inference on the frame
    results = model(frame)  # Asegúrate de tener tu modelo YOLOv8 definido previamente

    # Visualize the results on the frame
    annotated_frame = results[0].plot()

    # Display the annotated frame
    cv2.imshow("YOLOv8 Inference", annotated_frame)

    # Calcular el tiempo que lleva procesar este fotograma
    elapsed_time = time() - start_time

    # Calcular el retraso necesario para mantener una velocidad específica (por ejemplo, 30 FPS)
    delay = max(1, int((1 / target_fps - elapsed_time) * 1000))

    # Esperar el tiempo calculado
    cv2.waitKey(delay)

    # Actualizar el tiempo de referencia para el próximo fotograma
    start_time = time()

    # Si se presiona la tecla 'q', salir del bucle
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

In [ ]:
# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()

In [ ]:
#!pip uninstall opencv-python
#!pip uninstall ultralytics

In [ ]:
!conda list

In [ ]:
!pip install opencv-python

In [ ]:
!pip install opencv-python-headless

In [ ]:
# install https://www.msys2.org/
# install https://opencv.org/releases/
# install https://cmake.org/download/
# https://www.gtk.org/docs/installations/windows/

In [ ]:
#pacman -S mingw-w64-x86_64-pkg-config
#pacman -S mingw-w64-x86_64-gtk2
#pacman -S mingw-w64-x86_64-gtk4

In [ ]:
#!pip install cmake